# Data Management: substituting negative values with means

This notebook will show the algorithm used to substitute negative values found in the 'cases' and 'deaths' columns of our dataframe with the mean values of the country these negative values belong to. This algorithm works, however, because it is quite complex thus very slow in execution, it will be given only for demonstration purposes and not applied within the polynomial regression, as there we simply substituted the negative values with zeros. Unfortunately, due to time constraints and difficulties this algorithm could not have been improved.

The section titled **'Part 1'** will show the commented out slow and complex algorithm applied to all the countries, to substitute negative values with the mean.

The section titled **'Part 2'** will show a sample of this code running only for a country containing negative values. Spain was chosen as a sample country, as filtering through the data in our CSV file, we found that Spain contained two negative values in the 'deaths' column and one negative value in the 'cases' column.

In [31]:
# Importing the libraries to use.
import pandas as pd
import numpy as np
import datetime as dt

pd.options.mode.chained_assignment = None  # default='warn'

In [32]:
#Importing and printing the CSV dataset to work on.
dataset = "COVID-19-geographic-disbtribution-worldwide-2020-12-14.csv"
df = pd.read_csv(dataset)

# Extracting only the necessary columns to work on.
df = df[["dateRep", "cases", "deaths", "countriesAndTerritories"]]

 # Changing the date format of the dateRep column.
df["dateRep"]= pd.to_datetime(df["dateRep"])
df

,dateRep,cases,deaths,countriesAndTerritories
0,2020-12-14,746,6,Afghanistan
1,2020-12-13,298,9,Afghanistan
2,2020-12-12,113,11,Afghanistan
3,2020-12-11,63,10,Afghanistan
4,2020-12-10,202,16,Afghanistan
...,...,...,...,...
61895,2020-03-25,0,0,Zimbabwe
61896,2020-03-24,0,1,Zimbabwe
61897,2020-03-23,0,0,Zimbabwe
61898,2020-03-22,1,0,Zimbabwe


## Part 1

#### Creating a dataframe with the start and end indexes of each country within our main dataset.
Find the start and end index when rows with data for a specific country begin.

In [33]:
currentCountry = df['countriesAndTerritories'][0]
list_idx = []

i=0
start_idx = i
# Loop used to create the list to then transform into the dataframe.
# This list will contain the name of the countries along their start and end index within the main dataset used.
while i<len(df):
    if(df['countriesAndTerritories'][i] != currentCountry):
        list_idx.append(df['countriesAndTerritories'][i-1])
        list_idx.append(start_idx) # Start index
        list_idx.append(i-1) # End index
        #Assign new current country
        currentCountry = df['countriesAndTerritories'][i]
        #assign i to start_idx
        start_idx = i
        
    i+=1

# Appending to the list last set of values of the last country.
list_idx.append(currentCountry)
list_idx.append(start_idx)
list_idx.append(len(df)-1)
    
# Creating and printing the dataframe of the countries with their respective start and end index.
idx = np.array(list_idx).reshape(-1,3)
idx_df = pd.DataFrame(data=idx, columns=['Countries', 'Start index', 'End index'])
idx_df

,Countries,Start index,End index
0,Afghanistan,0,339
1,Albania,340,620
2,Algeria,621,965
3,Andorra,966,1241
4,Angola,1242,1509
...,...,...,...
209,Wallis_and_Futuna,60819,60877
210,Western_Sahara,60878,61110
211,Yemen,61111,61359
212,Zambia,61360,61630


#### Substitution of the country's negative values based on the mean of its data.
Commented out as explained above.

In [34]:
# # Substitute the negative values of a country with its specific mean value.
# sum_cases, sum_deaths, count_cases, count_deaths = 0, 0, 0, 0

# i, j = 0, 0
# while i<len(idx_df): #0 to 213
#     start = int(idx_df['Start index'][i]) # Storing the start index.
#     end = int(idx_df['End index'][i]) # Storing the end index.

#     # Slicing the main dataset's 'cases' column to have the specific values for a specific country.
#     lst_cases = list(df['cases'][start:end+1])
#     # Slicing the main dataset's 'deaths' column to have the specific values for a specific country.
#     lst_deaths = list(df['deaths'][start:end+1])
    
#     while j<len(lst_cases):
#         if(lst_cases[j] >= 0):
#             sum_cases += lst_cases[j]
#             count_cases += 1
        
#         if(lst_deaths[j] >= 0):
#             sum_deaths += lst_deaths[j]
#             count_deaths += 1
        
#         j+=1
    
#     # Substituing a country's negative cases and deaths values with their specific mean value calculated.
#     for z in range(len(df)):
#         if((df['countriesAndTerritories'][z] == idx_df['Countries'][i]) & (df['cases'][z] < 0)):
#             df['cases'][z] = sum_cases/count_cases
            
#         if((df['countriesAndTerritories'][z] == idx_df['Countries'][i]) & (df['deaths'][z] < 0)):
#             df['deaths'][z] = sum_deaths/count_deaths

#     sum_cases, sum_deaths, count_cases, count_deaths = 0, 0, 0, 0        
#     j=0
#     i += 1

# df.to_csv("prova.csv", index=False) # For verification purposes

## Part 2
Showing sample of above working code on Spain.

In [35]:
# Substitute the negative values of a country with its specific mean value.
sum_cases, sum_deaths, count_cases, count_deaths = 0, 0, 0, 0
start, end = 0, 0

country = "Spain" # Country to replace negative values.

# Finding the start and end index of the selected country from the main dataframe.
i = 0
while i<len(idx_df):
    if(idx_df['Countries'][i] == country):
        start = int(idx_df['Start index'][i])
        end = int(idx_df['End index'][i])
        break # Exit loop if Country is found.

    i+=1

# Slicing the main dataset's 'cases' column to have the specific values for the selected country.
lst_cases = list(df['cases'][start:end+1])
# Slicing the main dataset's 'deaths' column to have the specific values for the selected country.
lst_deaths = list(df['deaths'][start:end+1])


# Calculating the mean values for the country's deaths and cases.
j = 0
while j<len(lst_cases):
    if(lst_cases[j] >= 0):
        sum_cases += lst_cases[j]
        count_cases += 1

    if(lst_deaths[j] >= 0):
        sum_deaths += lst_deaths[j]
        count_deaths += 1

    j+=1

# Substituing the country's negative cases and deaths values with their specific mean value calculated.
i = start
while i<=end:
    if((df['countriesAndTerritories'][i] == country) & (df['cases'][i] < 0)):
        df['cases'][i] = sum_cases/count_cases
    
    if((df['countriesAndTerritories'][i] == country) & (df['deaths'][i] < 0)):
        df['deaths'][i] = sum_deaths/count_deaths
    
    i+=1

df.to_csv("verify_country.csv", index=False) # For verification purposes

Opening the **verify_country.csv** file and using the filter option in Excel to view all the negative values in the 'cases' and 'deaths' columns, we will see how Spain's negative deaths (-2 and -1918) and cases (-372) values will not be there as substituted with the mean value calculated from its cases and deaths. 